# Introduction




This notebook demonstrates caclulating the angular correlation of diffraction patterns recorded from an amorphous (or crystalline) material.

The dataset used for this demonstration is a 4-D STEM dataset of a PdNiP deposited thin film glass aquired using a DE-16 Camera and a 200keV FEI-Titan electron microscopt at 100 fps.  The probe size was ~2-nm and step size was .365 nm so there there is singificant probe overlap in the probe positions.

This functionality has been checked to run with pyxem-0.11.0 (May 2020). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

# Contents

1. <a href='#loa'> Importing & Visualization</a>
2. <a href='#s2'> Polar Reprojection</a>
3. <a href='#s3'> Angular Correlation</a>
4. <a href='#s4'> Power Spectrum and Correlation Maps</a>

 #  <a id='loa'></a> 1 - Importing and Visualization

In [ ]:
%matplotlib qt
import pyxem as pxm

In [ ]:
data = pxm.load("./data/09/PdNiP_test.hspy")

 #  <a id='s2'></a> 2 - Polar Reprojection
 
This section deals with converting the signal to a polar signal.  This is probably the most important and difficult part of the analysis.  Even small distortions in the pattern or misinterpertation of the center of the diffraction pattern will negitively affect the ability to determine correlations.

There is still some ongoing development on methods for identifying and correcting for these distortions but a good check is always to perform the correct and make sure that the first amorphous ring is a line after the polar reprojection.  In general your eye should be very good at identifying that.  Another thing to notice is that after the correlation if you have small splititing in all of your peaks(especially the self correlation) then most likely your center isn't completely correct.

In [ ]:
data.set_signal_type("electron_diffraction")
data.beam_energy=200
data.unit = "k_nm^-1"
rad = data.get_azimuthal_integral2d(npt_rad=100, center=(31.77734804, 31.23438963))

**Note:** This isn't perfect, as you can see there is still some distortion that an affine transformation could fix, but for the purposes of this demo this is sufficently good

In [ ]:
rad.sum().plot()

 #  <a id='s3'></a> 3 - Angular Correlations
 
This section deals with converting the signal to a correlation signal.  The most important part here is to properly mask the data.  This is important for example if you have a beam stop

In [ ]:
import numpy as np
mask = np.zeros((100,360), dtype=bool)

In [ ]:
mask[0:20,:]=True
mask[:,0:20]=True
mask[:,-20:]=True

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(mask)
plt.show()

In [ ]:
cor = rad.get_angular_correlation(mask = mask)

In [ ]:
cor.isig[:,3.0:5.0].plot()

 #  <a id='s4'></a> 4 - Power Spectrum and Correlation Maps
 
This section deals with visualization of the correlations as correlation maps.  These are spatial maps of the strucutre in some material. 

In [ ]:
power = cor.get_angular_power()

In [ ]:
f = plt.figure(figsize=(15,10))
power.plot_symmetries(k_region = [3.5,5.0],fig=f)